In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Familiar stack packages
import numpy as np
import lsst.geom as lsst_geom
import astropy.table as astTable
from lsst.daf.butler import Butler
import lsst.geom as geom
import xlens
from astropy.io import ascii as astascii
import fitsio
import glob

from astropy.visualization import ZScaleInterval
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import scipy.integrate as integrate
import lsst.afw.image as afwImage
import anacal

In [2]:
obs_repo = '/lustre/work/xiangchong.li/work/hsc_s23b_sim'
obs_collection = 'version1/image'

obs_butler = Butler(obs_repo, collections=obs_collection)
obs_registry = obs_butler.registry
skymap = obs_butler.get('skyMap', skymap="hsc")

In [3]:
full = fitsio.read("/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/tracts_fdfc_v1_trim2.fits")
entry = full[14]
tract_id = entry["tract"]
patch_db = entry["patch"]
patch_x = patch_db // 100
patch_y = patch_db % 100
patch_id = patch_x + patch_y * 9
print(tract_id, patch_id)

8280 73


In [4]:
%%time
db_dir = "/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/"
outdir = f"{db_dir}/s23b-anacal/tracts/{tract_id}/{patch_id}"
out_fname = os.path.join(outdir, "detect.fits")
patch_info = skymap[tract_id][patch_id]
wcs = patch_info.getWcs()
bbox = patch_info.getOuterBBox()
image_dir = (
    "/lustre/HSC_DR/hsc_ssp/dr4/s23b/data/s23b_wide/unified/deepCoadd_calexp"
)
files = glob.glob(os.path.join(image_dir, f"{tract_id}/{patch_id}/i/*"))
fname = files[0]
exposure = afwImage.ExposureF.readFits(fname)
seed = tract_id * 1000 + patch_id

CPU times: user 1.43 s, sys: 275 ms, total: 1.71 s
Wall time: 1.77 s


In [47]:
def resize_array(
    array,
    target_shape: tuple[int, int] = (64, 64),
    truth_catalog=None,
):
    """This is a util function to resize array to the target shape
    Args:
    array (NDArray): input array
    target_shape (tuple): output array's shape
    truth_catalog: truth catalog with image coordinates that need to be resized

    Returns:
    array (NDArray): output array with the target shape
    truth_catalog: resized truth catalog
    """
    target_height, target_width = target_shape
    input_height, input_width = array.shape

    # Crop if larger
    if input_height > target_height:
        start_h = (input_height - target_height) // 2
        array = array[start_h : start_h + target_height, :]
        if truth_catalog is not None:
            truth_catalog["image_y"] = truth_catalog["image_y"] - start_h
            truth_catalog["prelensed_image_y"] = (
                truth_catalog["prelensed_image_y"] - start_h
            )
    if input_width > target_width:
        start_w = (input_width - target_width) // 2
        array = array[:, start_w : start_w + target_width]
        if truth_catalog is not None:
            truth_catalog["image_x"] = truth_catalog["image_x"] - start_w
            truth_catalog["prelensed_image_x"] = (
                truth_catalog["prelensed_image_x"] - start_w
            )

    # Pad with zeros if smaller
    if input_height < target_height:
        pad_height = target_height - input_height
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        array = np.pad(
            array,
            ((pad_bottom, pad_top), (0, 0)),
            mode="constant",
            constant_values=0.0,
        )

    if input_width < target_width:
        pad_width = target_width - input_width
        pad_right = pad_width // 2
        pad_left = pad_width - pad_right
        array = np.pad(
            array,
            ((0, 0), (pad_left, pad_right)),
            mode="constant",
        )
    if truth_catalog is not None:
        return array, truth_catalog
    else:
        return array


def get_blocks(lsst_psf, lsst_bbox, lsst_mask, pixel_scale, npix):
    def make_circular_kernel(radius):
        """Create a binary circular (disk-shaped) kernel."""
        y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
        mask = x**2 + y**2 <= radius**2
        return mask.astype(np.int16)

    min_corner = lsst_bbox.getMin()
    # Get the x_min and y_min
    x_min = min_corner.getX()
    y_min = min_corner.getY()
    width, height = lsst_bbox.getWidth(), lsst_bbox.getHeight()

    if lsst_mask is not None:
        if "INEXACT_PSF" in lsst_mask.getMaskPlaneDict().keys():
            bitv = lsst_mask.getPlaneBitMask("INEXACT_PSF")
            mask_array = bitv & lsst_mask.array
            mask_array = (mask_array > 0).astype(np.int16)
        else:
            mask_array = np.zeros((height, width), dtype=np.int16)
    else:
        mask_array = np.zeros((height, width), dtype=np.int16)
    radius = 10
    kernel = make_circular_kernel(radius)
    mask_array = anacal.mask.sparse_convolve(mask_array, kernel)

    blocks = anacal.geometry.get_block_list(
        img_ny=height,
        img_nx=width,
        block_nx=300,
        block_ny=300,
        block_overlap=80,
        scale=pixel_scale,
    )

    for bb in blocks:
        x = max(min(bb.xcen, width - 15), 15)
        y = max(min(bb.ycen, height - 15), 15)
        for niter in range(5):
            if niter > 0:
                dx = np.random.randint(-niter, niter)
                dy = np.random.randint(-niter, niter)
            else:
                dx = 0
                dy = 0
            if mask_array[y+dy, x+dx] == 0:
                this_psf = lsst_psf.computeImage(
                    lsst_geom.Point2D(
                        x_min + x + dx,
                        y_min + y + dy
                    )
                ).getArray()
                bb.psf_array = resize_array(this_psf, (npix, npix))
                break

        if bb.psf_array.size == 0:
            found = False
            for j in range(max(bb.ymin, 0), min(width, bb.ymax)):
                if found:
                    break
                for i in range(max(bb.xmin, 0), min(height, bb.xmax)):
                    if mask_array[j, i] == 0:
                        this_psf = lsst_psf.computeImage(
                            lsst_geom.Point2D(x_min + i, y_min + j)
                        ).getArray()
                        bb.psf_array = resize_array(this_psf, (npix, npix))
                        found = True
                        break
    return blocks


In [49]:
lsst_bbox = exposure.getBBox()
lsst_mask = exposure.mask
lsst_psf = exposure.getPsf()
pixel_scale = 0.168
npix = 64
blocks = get_blocks(lsst_psf, lsst_bbox, lsst_mask, pixel_scale, npix)

In [53]:
for bb in blocks:
    print(bb.psf_array.shape)

(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(64, 64)
(

In [24]:
out[32, 32]

0.05002489799461351